In [91]:
import cv2
import os
import csv
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt
import time

In [92]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [93]:
def mediapipe_detection(frame, model):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return image, results

In [94]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [171]:
def record_landmarks(results, action):
    try:
        '''
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        right_hand = results.right_hand_landmarks.landmark
        right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())

        left_hand = results.left_hand_landmarks.landmark
        left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
        '''

        if results.pose_landmarks:
            pose_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten())
        else:
            pose_row = list(np.zeros(132).flatten())
        
        if results.left_hand_landmarks:
            lh_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten())
        else:
            lh_row = list(np.zeros(84).flatten())
        
        if results.right_hand_landmarks:
            rh_row = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten())
        else:
            rh_row = list(np.zeros(84).flatten())
        
        row = pose_row + right_hand_row + left_hand_row
        row.insert(0, action)

        with open('coords.csv', mode='a', newline='') as f:
            csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer_obj.writerow(row)
            
    except:
        pass        

In [124]:
len(results.pose_landmarks.landmark)*4

132

# Testing Camera and recording

In [175]:
#TESTING CAMERA

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

In [26]:
cam.release()
cv2.destroyAllWindows()

In [179]:
#CREATE HEADER IN CSV
NUM_COORDS = 75
#num_coords = len(results.pose_landmarks.landmark) + len(results.right_hand_landmarks.landmark) + len(results.left_hand_landmarks.landmark)
header_list = ["action"]
for i in range(1, NUM_COORDS+1):
    header_list += ["x{}".format(i),"y{}".format(i),"z{}".format(i),"v{}".format(i)]

with open('coords.csv', mode='w', newline='') as f:
    csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer_obj.writerow(header_list)

In [290]:
#TEST
NUM_COORDS = 75
#num_coords = len(results.pose_landmarks.landmark) + len(results.right_hand_landmarks.landmark) + len(results.left_hand_landmarks.landmark)
header_list = ["action"]
for i in range(1, NUM_COORDS+1):
    header_list += ["x{}".format(i),"y{}".format(i),"z{}".format(i),"v{}".format(i)]

with open('coordsTESTadd.csv', mode='w', newline='') as f:
    csv_writer_obj = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer_obj.writerow(header_list)

In [195]:
DATA_PATH = os.path.join('MP_data')
actions = np.array(["CallPolice", "Emergency"])
no_sequences = 30
sequence_length = 120

In [196]:
#recording per sequence

cam = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_no in range(sequence_length):
                
                ret, frame = cam.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_landmarks(image, results)
                
                #print(results.right_hand_landmarks)
                #extract_keypoints(results)
                if frame_no == 0:
                    cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                     cv2.putText(image, "Collecting Video for {} Video number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

                record_landmarks(results, action)
                #npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_no))
                #np.save(npy_path, keypoints)
                
                cv2.imshow("Input Sign Language", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cam.release()
                    cv2.destroyAllWindows()

    cam.release()
    cv2.destroyAllWindows()

In [60]:
#RECORDING WITHOUT ANY SEQUENCE

cam = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)

        record_landmarks(results)
        
        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #print(results.right_hand_landmarks) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

# SPLIT DATA

In [260]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [293]:
df = pd.read_csv("coordsTESTadd.csv")

In [294]:
X = df.drop('action', axis=1) #feature
y = df['action'] #target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1234)

In [295]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score
import pickle


In [296]:
#TRAINING

pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [298]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [299]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [303]:
fit_models['rf'].predict(X_test)

array(['Hurt', 'Ambulance', 'Ambulance', 'Doctor', 'Ambulance', 'Mom',
       'Hurt', 'Mom', 'Ambulance', 'Doctor', 'Mom', 'Mom', 'Mom',
       'Doctor', 'Doctor', 'Allergy', 'Doctor', 'Doctor', 'Mom',
       'Allergy', 'Doctor', 'Allergy', 'Hurt', 'Hurt', 'Mom', 'Mom',
       'Doctor', 'Allergy', 'Allergy', 'Mom', 'Allergy', 'Hurt', 'Mom',
       'Mom', 'Hurt', 'Hurt', 'Mom', 'Ambulance', 'Hurt', 'Hurt', 'Mom',
       'Allergy', 'Mom', 'Hurt', 'Mom', 'Hurt', 'Allergy', 'Hurt', 'Hurt',
       'Allergy', 'Ambulance', 'Ambulance', 'Ambulance', 'Ambulance',
       'Doctor', 'Hurt', 'Doctor', 'Hurt', 'Allergy', 'Doctor',
       'Ambulance', 'Mom', 'Mom', 'Ambulance', 'Ambulance', 'Allergy',
       'Doctor', 'Allergy', 'Ambulance', 'Doctor', 'Ambulance',
       'Ambulance', 'Mom', 'Ambulance', 'Doctor', 'Allergy', 'Doctor',
       'Doctor', 'Ambulance', 'Allergy', 'Ambulance', 'Doctor', 'Hurt',
       'Doctor', 'Ambulance', 'Ambulance', 'Hurt', 'Ambulance',
       'Ambulance', 'Allergy', 'M

In [304]:
y_test

78           Hurt
366     Ambulance
351     Ambulance
755        Doctor
564     Ambulance
          ...    
183          Hurt
1294      Allergy
188          Hurt
889        Doctor
0            Hurt
Name: action, Length: 150, dtype: object

In [289]:
with open('body_languageTEST.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [302]:
with open('body_languageTEST.pkl', 'rb') as f:
    model = pickle.load(f)

In [328]:
model.predict(X_test)

array(['Hurt', 'Ambulance', 'Ambulance', 'Doctor', 'Ambulance', 'Mom',
       'Hurt', 'Mom', 'Ambulance', 'Doctor', 'Mom', 'Mom', 'Mom',
       'Doctor', 'Doctor', 'Allergy', 'Doctor', 'Doctor', 'Mom',
       'Allergy', 'Doctor', 'Allergy', 'Hurt', 'Hurt', 'Mom', 'Mom',
       'Doctor', 'Allergy', 'Allergy', 'Mom', 'Allergy', 'Hurt', 'Mom',
       'Mom', 'Hurt', 'Hurt', 'Mom', 'Ambulance', 'Hurt', 'Hurt', 'Mom',
       'Allergy', 'Mom', 'Hurt', 'Mom', 'Hurt', 'Allergy', 'Hurt', 'Hurt',
       'Allergy', 'Ambulance', 'Ambulance', 'Ambulance', 'Ambulance',
       'Doctor', 'Hurt', 'Doctor', 'Hurt', 'Allergy', 'Doctor',
       'Ambulance', 'Mom', 'Mom', 'Ambulance', 'Ambulance', 'Allergy',
       'Doctor', 'Allergy', 'Ambulance', 'Doctor', 'Ambulance',
       'Ambulance', 'Mom', 'Ambulance', 'Doctor', 'Allergy', 'Doctor',
       'Doctor', 'Ambulance', 'Allergy', 'Ambulance', 'Doctor', 'Hurt',
       'Doctor', 'Ambulance', 'Ambulance', 'Hurt', 'Ambulance',
       'Ambulance', 'Allergy', 'M

In [324]:
right_hand_test = list(np.zeros(84).flatten())
left_hand_test = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten())

In [326]:
len(right_hand_test)

84

# Real Time Testing

In [305]:
cam = cv2.VideoCapture(0)

threshold = 0.4

with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
        ret, frame = cam.read()
    
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        
        #print(results.right_hand_landmarks)

        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
    
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
    
            left_hand = results.left_hand_landmarks.landmark
            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
     
            row = pose_row + right_hand_row + left_hand_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            '''   coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)'''

            
            # Get status box
            cv2.rectangle(image, (0,0), (640, 35), (0,0,0, 0.5), -1)

            cv2.putText(image, 'Action:'
                        , (10,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (110,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'Accuracy:'
                        , (420,25), cv2.FONT_HERSHEY_DUPLEX, 0.85, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (560,25), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
        except:
            pass


        cv2.imshow("Sign Language Converter", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Hurt [0.03282363 0.34080681 0.00197722 0.60336346 0.02102888]
Hurt [0.01265514 0.32027456 0.00308235 0.62793721 0.03605075]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Ambulance [0.07876586 0.47727551 0.00449156 0.41162951 0.02783757]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


...

Ambulance [0.19616887 0.56227166 0.19718666 0.01358874 0.03078407]
Ambulance [0.29217852 0.55369896 0.10607494 0.01301278 0.0350348 ]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Ambulance [0.28295714 0.53299599 0.12239817 0.01465913 0.04698958]
Ambulance [0.35297547 0.49223486 0.08985185 0.01889249 0.04604532]


C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\Karthik\.conda\envs\condamp\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


Ambulance [0.28817013 0.55163968 0.10156638 0.02059482 0.03802899]
Ambulance [0.26577482 0.50019813 0.19807699 0.02689934 0.00905073]
